In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import random
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
data_products_prior = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/order
ordersDF = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/orders.csv")
productsDF = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/products.csv")
order_products_train = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/orders.csv")
order_products_prior = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/orders.csv")

In [ ]:
print("The order_products_train size is : ", order_products_train.shape)
print("The order_products_prior size is : ", order_products_prior.shape)

In [ ]:
order_products_prior.head(2)

In [ ]:
order_products_all = pd.concat([order_products_train, order_products_prior], axis=0)
print("The order_products_all size is : ", order_products_all.shape)

In [ ]:
order_products_all.head(5)

In [ ]:
total = order_products_all.isnull().sum().sort_values(ascending=False)
percent = (order_products_all.isnull().sum()/order_products_all.isnull().count()).sort_valu
missing_data = pd.concat([total, percent], axis=1, keys=['Total Missing', 'Percent'])
missing_data

In [ ]:
#the most ordered products
grouped = order_products_all.groupby("product_id")["reordered"].aggregate({'Total_reorders'
grouped = pd.merge(grouped, productsDF[['product_id', 'product_name']], how='left', on=['pr
grouped = grouped.sort_values(by='Total_reorders', ascending=False)[:10]
grouped

In [ ]:
grouped = ordersDF.groupby("eval_set")["order_id"].aggregate({'Total_orders': 'count'}).res
grouped['Ratio'] = grouped["Total_orders"].apply(lambda x: x /grouped['Total_orders'].sum()
grouped

# Data Visualization

In [ ]:
#unique DOW values
ordersDF.order_dow.unique()
#what is the frequency of the orders according to days
n, bins, patches = plt.hist(ordersDF.order_dow, 13, facecolor="red", alpha=.75, align='mid'
plt.xlabel("Day of Week")
plt.ylabel("Orders Count")
plt.title("When do people buy?")
plt.show()

In [ ]:
ordersDF.order_hour_of_day.unique()
n, bins, patches = plt.hist(ordersDF.order_hour_of_day, 47, facecolor="green", alpha=.75, a
plt.xlabel("Hour of Day")
plt.ylabel("Orders Count")
plt.title("When do people buy in a Day?")
plt.show()

In [ ]:
grouped = ordersDF.groupby("eval_set")["user_id"].apply(lambda x: len(x.unique()))
plt.figure(figsize=(7,8))
sns.barplot(grouped.index, grouped.values, palette='bright')
plt.ylabel('Number of users', fontsize=15)
plt.xlabel('Eval set', fontsize=15)
plt.title("Number of unique customers in each dataset")
plt.show()

In [ ]:
departmentsDF = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/departments
aislesDF = pd.read_csv("C:/Users/Harsh Patel/Desktop/Big Data Mini Project/aisles.csv")

In [ ]:
aislesDF.head(7)

In [ ]:
#Merge in single dataframe
items = pd.merge(left =pd.merge(left=productsDF, right=departmentsDF, how='left'), right=a
items.head()

In [ ]:
#Most important departments
grouped = items.groupby("department")["product_id"].aggregate({'Total_products': 'count'}).
grouped['Ratio'] = grouped["Total_products"].apply(lambda x: x /grouped['Total_products'].s
grouped.sort_values(by='Total_products', ascending=False, inplace=True)
grouped

# Recommendation Model

In [ ]:
#Merging Datasets
opt = order_products_train.merge(productsDF,how='left', on='product_id')
opt = opt.merge(departmentsDF,how='left', on='department_id')
opt = opt.merge(aislesDF,how='left', on='aisle_id')

In [ ]:
#Filtering of Data by reorders
reorders = opt[opt['reordered'] == 1]
reorders['product_id'] = reorders['product_id'].astype('int64')

In [ ]:
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
[reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.t

In [ ]:
# mask the reorders dataframe to only incluse dem hi vol prods
reorders = reorders[reorders['product_id'].isin(hivol)]

In [ ]:
#filters the High demand items greater than.
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)

In [ ]:
hidem_ord = reorders[reorders['hi_dem'] == True]

In [ ]:
user_orders = reorders.merge(ordersDF)

In [ ]:
hidem_ord = user_orders[user_orders['hi_dem'] == True]

In [ ]:
#return the total items
users = hidem_ord.groupby(['user_id','product_name']).size().sort_values(ascending=False).u

In [ ]:
#creates a similiarity by users.
users_sim = pd.DataFrame(cosine_similarity(users),index=users.index,columns=users.index)

In [ ]:
def next_prod(df,num_col):
return df[df.columns[num_col]].drop(df.columns[num_col]).sort_values(ascending=False).h

In [ ]:
#returns similar users to this one.
pd.DataFrame(next_prod(users_sim,56)).T

In [ ]:
#gives a recommendation for the last product added to shopping cart
pd.DataFrame(next_prod(products_sim,11)).T

In [ ]:
#Product Recommender by Order id
baskets = hidem_ord.groupby(['product_name','order_id']).size().sort_values(ascending=False
basket_sim = pd.DataFrame(cosine_similarity(baskets),columns=baskets.index,index=baskets.index
basket_sim['Green Peas'].sort_values(ascending=False).head(10)[1:]